In [14]:
import pandas as pd
import konlpy

data = pd.read_csv("sample_text.csv", header=None, names=['text'])
data.head()

,text
0,인공지능 또는 AI는 인간의 학습능력 추론능력 지각능력 그외에 인공적으로 구현한 컴...
1,하나의 인프라 기술이기도 하다
2,인간을 포함한 동물이 갖고 있는 지능 즉 natural intelligence와는 ...
3,지능을 갖고 있는 기능을 갖춘 컴퓨터 시스템이며 인간의 지능을 기계 등에 인공적으로...
4,일반적으로 범용 컴퓨터에 적용한다고 가정한다


In [15]:
tokenizer = konlpy.tag.Mecab()
tokenizer.morphs("안녕하세요")

['안녕', '하', '세요']

In [16]:
data['text'] = data['text'].str.replace(",", "").replace(".", "")
data['morphs'] = data['text'].map(lambda x: tokenizer.morphs(x))
data

,text,morphs
0,인공지능 또는 AI는 인간의 학습능력 추론능력 지각능력 그외에 인공적으로 구현한 컴...,"[인공지능, 또는, AI, 는, 인간, 의, 학습, 능력, 추론, 능력, 지각, 능..."
1,하나의 인프라 기술이기도 하다,"[하나, 의, 인프라, 기술, 이, 기, 도, 하, 다]"
2,인간을 포함한 동물이 갖고 있는 지능 즉 natural intelligence와는 ...,"[인간, 을, 포함, 한, 동물, 이, 갖, 고, 있, 는, 지능, 즉, natur..."
3,지능을 갖고 있는 기능을 갖춘 컴퓨터 시스템이며 인간의 지능을 기계 등에 인공적으로...,"[지능, 을, 갖, 고, 있, 는, 기능, 을, 갖춘, , 컴퓨터, 시스템, 이,..."
4,일반적으로 범용 컴퓨터에 적용한다고 가정한다,"[일반, 적, 으로, 범용, 컴퓨터, 에, 적용, 한다고, 가정, 한다]"
5,이 용어는 또한 그와 같은 지능을 만들 수 있는 방법론이나 실현 가능성 등을 연구하...,"[이, 용어, 는, 또한, 그, 와, 같, 은, 지능, 을, 만들, 수, 있, 는,..."


In [28]:
word_set = set()

data['morphs'].map(lambda x: word_set.update(set(x)))

word_set

{'AI',
 'intelligence',
 'natural',
 '\xa0',
 '가능',
 '가정',
 '갖',
 '갖춘',
 '같',
 '개념',
 '것',
 '고',
 '과학',
 '구현',
 '그',
 '기',
 '기계',
 '기능',
 '기술',
 '는',
 '능력',
 '다',
 '다른',
 '도',
 '동물',
 '등',
 '또는',
 '또한',
 '를',
 '만들',
 '며',
 '방법론',
 '범용',
 '분야',
 '성',
 '수',
 '시스템',
 '시연',
 '실현',
 '에',
 '연구',
 '와',
 '외',
 '용어',
 '으로',
 '은',
 '을',
 '의',
 '이',
 '이나',
 '인간',
 '인공',
 '인공지능',
 '인프라',
 '일반',
 '있',
 '적',
 '적용',
 '즉',
 '지각',
 '지능',
 '지칭',
 '추론',
 '컴퓨터',
 '포함',
 '프로그램',
 '하',
 '하나',
 '학습',
 '한',
 '한다',
 '한다고'}

In [29]:
chars = set([w_i for w in word_set for w_i in w])
chars = sorted(list(chars))
print(chars)

['A', 'I', 'a', 'c', 'e', 'g', 'i', 'l', 'n', 'r', 't', 'u', '\xa0', '가', '각', '간', '갖', '같', '개', '것', '계', '고', '공', '과', '구', '그', '기', '나', '념', '는', '능', '다', '도', '동', '들', '등', '또', '라', '램', '력', '로', '론', '른', '를', '만', '며', '물', '반', '방', '범', '법', '분', '성', '수', '술', '스', '습', '시', '실', '야', '어', '에', '연', '와', '외', '용', '으', '은', '을', '의', '이', '인', '일', '있', '적', '정', '즉', '지', '추', '춘', '칭', '컴', '터', '템', '포', '퓨', '프', '하', '학', '한', '함', '현']
